In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Fullscreen
#import seaborn
import holoviews
import hvplot.pandas 
import panel as pn
import matplotlib as plt
%matplotlib inline
pn.extension('tabulator')
#from panel import theme
#from IPython.display import display
#import os

In [ ]:
stats = pd.read_csv("data/stats_merged.csv",encoding="utf-8") # social statistics
admin = gpd.read_file("data/Stadtteile_Hamburg.shp",crs="EPSG:25832") # boundary dataset 
noise = pd.read_csv("data/admin_noise.csv",encoding="utf-8") # social statistics

#combined_indicators = pd.read_csv("data/combined_indicators.csv",encoding="utf-8") # social statistics

In [ ]:
#drop unrelevant columns
stats.drop(columns=['geometry_x','geometry_y', 'geometry','bezirk_nam_x', 'bezirk_nam_y', 'ZONE_CODE', 'COUNT', 'AREA', 'MIN','MAX', 'RANGE'], inplace=True);


In [ ]:
admin_stats = pd.merge(admin, stats, left_index=True, right_index=True) 


In [ ]:
"""-----Function to add a status column  --- """

def add_status_column(df, perc_col): #The function will be used for all of the different indicators)
    """The function adds a text column to the dataframe. The value of the text column is assigned based
        on the values of the classified "_percentile column"

    Parameters: 
        df : dataframe used for the classification
        perc_col: Column used to assign te the text values: NEEDS to be a categorial variable, resulting from the 
        "percentile functions" Has to take the classified column:result of func percentile as input
        
    Returns:
        A dataframe with a new column called "status" containing strings with the values from "very low" to "very high"
        Assignes the names based on the values in the perc_col
        
    Apply the functin:
        Example: add_status_column(dataframe, 'columnName') # 

    """
    df['status'] = ['very low' if x == 0 else  #creates the new column "status and then adds text description to the corresponding value"
                   'low' if x == 1 else
                   'medium' if x == 2 else
                   'high' if x == 3 else
                   'very high' for x in df[perc_col]]

    return df.head()

In [ ]:
add_status_column(admin_stats, 'combined_indicators_percentile') #add a new column to the frame

In [ ]:
#subset dataframe
stats_to_plot = admin_stats[['district','geometry','qt_soc_stats','status','weighted_area_percentile', 'z_area_per_inhbt_percentile','MEDIAN_percentile','env_multiple_burden_percentile', 'combined_indicators_percentile']]

In [ ]:
#rename
stats_to_plot_rename=stats_to_plot.rename(columns={"district":"District","weighted_area_percentile":"Noise Pollution","z_area_per_inhbt_percentile":"Green Area Supply ","MEDIAN_percentile":"Thermal Burden","env_multiple_burden_percentile":"Combined Environmental Burden" , "combined_indicators_percentile":"Combined Indicators", "status":"Integrated Environmental & Social Status Index","qt_soc_stats":"Social Status"})

In [ ]:
#drop NaN and reproj to Hamburg CRS
stats_to_plot_rename.dropna(inplace=True);
stats_to_plot_reproj=stats_to_plot_rename.to_crs(epsg=25832)

In [ ]:
stats_to_plot_reproj.columns

In [ ]:
#create folium map
layer_name = 'Classified Indicators'
layer_noise ="NoisePolution"
m = folium.Map(location=[53.55, 9.99], zoom_start=14)

m = stats_to_plot_reproj.explore('Integrated Environmental & Social Status Index', # show the Distance column
                   cmap='RdPu', name=layer_name# use the 'plasma' colormap from matplotlib
                  )




folium.TileLayer(tiles="CartoDB positron", overlay=True).add_to(m)

#folium.TileLayer("CartoDB positron").add_to(m)
folium.TileLayer("OpenStreetMap").add_to(m)
folium.TileLayer("Stamen Terrain").add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
stats_grouped = admin_stats.dissolve(by='bezirk_nam');

In [ ]:
# subset dataframe without geometry column to enable plotting in hvplot
stats_reduced = admin_stats[['district', 'bezirk_nam','z_area_per_inhbt','qt_soc_stats','%unemployed', '%social_benefits','%social_housing', 'weighted_area_percentile', 'z_area_per_inhbt_percentile','MEDIAN_percentile','env_multiple_burden_percentile', 'combined_indicators_percentile', 'z_perc_green_area', 'green_space_count','weighted_area']]
stats_reduced=stats_reduced.rename(columns={"district":"District","bezirk_nam":"Admin1","z_area_per_inhbt":"green area per inhabitant(sqm)","weighted_area_percentile":"Noise Pollution","z_area_per_inhbt_percentile":"Green Area Supply" ,"MEDIAN_percentile":"Thermal Burden","env_multiple_burden_percentile":"Multiple Environmental Burden", "combined_indicators_percentile":"Combined Indicators","z_perc_green_area":"%green area per district", "green_space_count":"Count of Green Areas per District","weighted_area":"noise affected housing area","qt_soc_stats":"Social Status"})

In [ ]:
stats_reduced_I = stats_reduced.interactive # makes dataframe interactive

In [ ]:
#scatter plot
x = pn.widgets.Select(name='Social Statistics', options=['%social_housing', '%unemployed', '%social_benefits'])
y = pn.widgets.Select(name='Environmental Statistics', options=['green area per inhabitant(sqm)', '%green area per district', 'noise affected housing area'])
#kind = pn.widgets.Select(name='kind', value='scatter', options=['kde', 'scatter'])

#plot = stats_reduced.hvplot(x=x, y=y, kind=kind, colorbar=False, width=600,by="bezirk_nam" )
plot_scatter = stats_reduced_I.hvplot(x=x, y=y, kind='scatter', colorbar=False, width=600,alpha=0.7,by="Admin1")

plot_scatter_layout = pn.Column(
    pn.pane.Markdown("### Select combinations of different variables to show scatter plot", styles={'font-weight': 'bold', 'color':'grey'}),
    plot_scatter
    
)

plot_scatter_layout

In [ ]:
# Get the unique district names
district_names = list(stats_reduced['District'].unique())

# Create the dropdown widget for selecting the district
district_dropdown = pn.widgets.Select(name='Select District', options=district_names)

# Define a callback function for filtering the data based on the selected district
def filter_data(event):
    selected_district = district_dropdown.value
    filtered_data = stats_reduced[stats_reduced['District'] == selected_district]
    stats_table.value = filtered_data

# Register the callback function with the dropdown widget
district_dropdown.param.watch(filter_data, 'value')

# Create the interactive table
stats_table = pn.widgets.Tabulator(stats_reduced, pagination='remote', page_size=10, sizing_mode='stretch_width')

# Create the "Reset" button
reset_button = pn.widgets.Button(name='Reset')

# Define a callback function for resetting the filter
def reset_filter(event):
    district_dropdown.value = None
    stats_table.value = stats_reduced

# Register the callback function with the "Reset" button
reset_button.on_click(reset_filter)
# Create the layout with the dropdown and table
table_layout = pn.Column(district_dropdown, reset_button, stats_table)

# Display the layout
table_layout

In [ ]:
status_counts = stats.groupby('district')[['status_noise', 'status_greens', 'status_soc','env_multiple_burden','combined_indicators']].apply(lambda x: x.apply(pd.Series.value_counts)).unstack().fillna(0)


In [ ]:
# Radio buttons maps
indicators_radio = pn.widgets.RadioButtonGroup(
    name='map', 
    options=['Green', 'Noise'],
    button_type='success'
)

indicators_radio